In [ ]:
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi2
except:
  IN_COLAB = False

In [ ]:
import pandas as pd
from jax import numpy as jnp
from typing import Dict
import copy

from summer2 import CompartmentalModel
from summer2.parameters import Parameter as param
from summer2.parameters import Function as func
from summer2.parameters import Time

pd.options.plotting.backend = "plotly"

## Crude birth flow

New people are born into the destination compartment every time unit (also referred to as recruitment).
A "crude birth rate" is multiplied through by the population size to calculate the rate of entry into the starting population.
Therefore, this is the per capita rate of new births for the entire population.

Other approaches are sometimes used in epidemiology,
such as linking the rate of new births to the number of females of child-bearing age in the population (fertility rate).
Implementing such an approach would require us to model this sub-population.

In [ ]:
def get_sir_with_birth_model(
    parameters: Dict
)-> CompartmentalModel:
    """
    Take the previous SIR model and add a birth flow.
    
    Args:
        parameters: The parameter values to be used in running the model
    Returns:
        The summer model object
    """
    
    model = get_sir_freq_model(parameters)

    # Add crude birth flow to the model
    model.add_crude_birth_flow(
        "crude_birth", 
        birth_rate=param("birth"), 
        dest="susceptible"
    )
    
    return model

In [ ]:
sir_params.update({"birth": 0.03})  # Add an arbitrary crude birth rate

sir_birth_model = get_sir_with_birth_model(sir_params)
    
sir_birth_model.run(parameters=sir_params)
compartment_values = sir_birth_model.get_outputs_df()
compartment_values.plot()

Clearly the total size of the population is growing. The rate of births here (0.03) is very high if we think of the time unit as years, and implausible if we think of it as days, but the purpose is to illustrate the effect of this process. 

## Replacement birth flow

Add a flow to replace the number of deaths into the destination compartment. This means the total population should be conserved over time.

In [ ]:
def get_sir_with_birth_replace_model(
    parameters: Dict
)-> CompartmentalModel:
    """
    Take the previous SIR model with infection-specific dceaths included,
    and add a birth flow to replace these deaths.
    
    Args:
        parameters: The parameter values to be used in running the model
    Returns:
        The summer model object
    """
    
    model = get_sir_with_death_model(parameters)
    
    # Add replacement birth flow to the model
    model.add_replacement_birth_flow(
        "crude_birth", 
        dest="susceptible"
    )
    
    return model

In [ ]:
sir_birth_replace_model = get_sir_with_birth_replace_model(sir_params)

sir_birth_replace_model.run(parameters=sir_params)
compartment_values = sir_birth_replace_model.get_outputs_df()
compartment_values.plot()

In [ ]:
sir_birth_replace_model = get_sir_with_birth_replace_model(sir_params)

sir_birth_replace_model.run(parameters=sir_params)
compartment_values = sir_birth_replace_model.get_outputs_df()
compartment_values.plot()